### The purpose of this notebook is to showcase how to do association rule mining

* we will need to install mlxtend:
 - conda install -c conda-forge mlxtend

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from datetime import datetime, timedelta
import collections

from util_transaction_data import *

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)


In [2]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.preprocessing import *
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpmax

 - first, prepare the data

In [3]:
filename = './AwanTunai_transaction_data.csv'
baskets = pd.read_csv(filename)
baskets.dropna(inplace=True)
baskets["spent"] = baskets["qty"] * baskets["price"] # column wide multiplication
baskets = add_date_cols(baskets)
merchant_attributes = get_merchant_attributes(baskets)
sku_atributes = get_sku_attributes(baskets)
top_cat_attributes = get_top_cat_attributes(baskets)

# baskets.groupby('order_id').groups.values()

In [4]:
transaction_data = []
for groups in baskets.groupby('order_id').groups.values():
    transaction_data.append(list(baskets.loc[groups]['sku_id'].to_numpy()))
transaction_data[:10]

[[10, 6, 14, 5],
 [15, 9],
 [11, 13, 27, 7, 26, 30, 23, 28, 19],
 [7, 26, 32],
 [4, 25, 31, 14, 28, 17, 18],
 [27],
 [26, 10, 4],
 [31],
 [9, 2],
 [10, 15, 1, 14, 6, 8]]

In [5]:
def oneHotCoding(transaction_data):
    te = TransactionEncoder()
    te_ary = te.fit(transaction_data).transform(transaction_data)
    te_df = pd.DataFrame(te_ary, columns=te.columns_)
    return te_df

transaction_oneHot = oneHotCoding(transaction_data)

In [6]:
transaction_oneHot

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,...,1574,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1603,1604,1605,1606,1607,1608,1609,1610,1611,1612,1613,1614,1615,1616,1617,1618,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1630,1631,1632,1633,1634,1635,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682,1683,1684,1685,1686,1687,1688,1689,1690,1691,1692,1693,1694,1695,1696,1697,1698,1699,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709,1710,1711,1712,1713,1714,1715,1716,1717,1718,1719,1720,1721,1722,1723,1724,1725,1726,1727,1728,1729,1730,1731,1732,1733,1734,1735,1736,1737,1738,1739,1740,1741,1742,1743,1744,1745,1746,1747,1748,1749,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,1760,1761,1762,1763,1764,1765,1766,1767,1768,1769,1770,1771,1772,1773,1774,1775,1776,1777,1778,1779,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815,1816,1817,1818,1819,1821,1822,1823,1824,1825
0,False,False,False,False,True,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [28]:
apriori(transaction_oneHot, min_support=0.02)

,support,itemsets
0,0.032931,(128)
1,0.026903,(132)
2,0.035871,(179)
3,0.026922,(183)
4,0.021624,(185)
...,...,...
433,0.020276,"(984, 988, 1502, 991)"
434,0.023477,"(984, 988, 1502, 1511)"
435,0.025368,"(984, 1511, 1502, 991)"
436,0.022391,"(1511, 988, 1502, 991)"


 - double check with frequent count of numer of order per SKU

In [8]:
sku_atributes.num_orders.sort_values(ascending = False).head(10)

1502    15119
866     11296
1511    10666
873      9772
971      9482
984      9469
970      8008
991      7365
1337     7329
972      7026
Name: num_orders, dtype: int64

In [9]:
apriori(transaction_oneHot, min_support=0.15)

,support,itemsets
0,0.211480,(866)
1,0.182948,(873)
2,0.177519,(971)
3,0.177276,(984)
4,0.283053,(1502)
5,0.199685,(1511)


In [10]:
min_freq = 200
min_support = min_freq/transaction_oneHot.shape[0]
min_confidence = 0.6
min_rule_support = 0.2
min_lift = 0.15

In [11]:
freq_itemsets_apriori = apriori(transaction_oneHot,min_support=min_support,use_colnames=True)
freq_itemsets_apriori.describe()

,support
count,11885.000000
mean,0.007509
std,0.008577
min,0.003744
25%,0.004306
50%,0.005317
75%,0.007620
max,0.283053


In [12]:
freq_itemsets_fpgrowth = fpgrowth(transaction_oneHot,min_support=min_support,use_colnames=True)
freq_itemsets_fpgrowth.describe()

,support
count,11885.000000
mean,0.007509
std,0.008577
min,0.003744
25%,0.004306
50%,0.005317
75%,0.007620
max,0.283053


In [13]:
freq_itemsets_fpmax = fpmax(transaction_oneHot,min_support=min_support,use_colnames=True)
freq_itemsets_fpmax.describe()

,support
count,5035.000000
mean,0.004413
std,0.000766
min,0.003744
25%,0.003950
50%,0.004212
75%,0.004606
max,0.010896


In [14]:
# compute and print the association rules
def basket_rules(freq_itemsets,metrics,threshold):
    asso_rules = association_rules(freq_itemsets, metric=metrics, min_threshold=threshold)
    return asso_rules.sort_values(by='lift', ascending=False)[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

rules_apriori = basket_rules(freq_itemsets_apriori,"confidence",0.60)
rules_fpgrowth = basket_rules(freq_itemsets_fpgrowth,"confidence",0.60)


In [15]:
rules_fpgrowth

,antecedents,consequents,support,confidence,lift
21376,"(857, 979)","(1208, 873)",0.003857,0.611276,55.813153
178,"(129, 186, 412)","(133, 406)",0.003819,0.641509,37.003871
157,"(412, 186, 180)","(133, 406)",0.004025,0.630499,36.368735
199,"(129, 186, 180)","(133, 406)",0.004643,0.600484,34.637437
200,"(186, 180, 133)","(129, 406)",0.004643,0.650919,31.607425
...,...,...,...,...,...
7835,"(977, 874, 1181)",(1504),0.006290,0.601073,2.123535
3374,"(1184, 977)",(1504),0.006534,0.600688,2.122176
9633,"(928, 977)",(1504),0.007433,0.600605,2.121881
4988,"(973, 1095)",(1504),0.005766,0.600390,2.121121


In [34]:
rules_fpmax

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [16]:
def predict(antecedent, rules, consequents_only = False):
    
    # get the rules for this antecedent
    preds = rules[rules['antecedents'] == antecedent]
    if consequents_only:
        # a way to convert a frozen set with one element to string
        preds = preds['consequents'].apply(iter).apply(next)
    return preds

In [35]:
rules_fpmax = association_rules(freq_itemsets_fpmax, metric="confidence", min_threshold=0.001, support_only=True)

In [18]:
min_support

0.0037443366907552325

In [36]:
predict({1786}, rules_fpmax, consequents_only=False)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1151,(1786),(928),NaN,NaN,0.003763,NaN,NaN,NaN,NaN
1153,(1786),(992),NaN,NaN,0.004662,NaN,NaN,NaN,NaN
1158,(1786),"(1504, 867)",NaN,NaN,0.003782,NaN,NaN,NaN,NaN
1160,(1786),(1339),NaN,NaN,0.004868,NaN,NaN,NaN,NaN
1167,(1786),"(1504, 874)",NaN,NaN,0.003950,NaN,NaN,NaN,NaN
1172,(1786),"(1338, 973)",NaN,NaN,0.003744,NaN,NaN,NaN,NaN
1177,(1786),"(972, 973)",NaN,NaN,0.003932,NaN,NaN,NaN,NaN
1184,(1786),"(1504, 972)",NaN,NaN,0.004100,NaN,NaN,NaN,NaN
1190,(1786),"(989, 973)",NaN,NaN,0.003969,NaN,NaN,NaN,NaN
1196,(1786),"(1504, 989)",NaN,NaN,0.004081,NaN,NaN,NaN,NaN


In [20]:
predict({866}, rules_fpgrowth)


,antecedents,consequents,support,confidence,lift
424,(866),(971),0.052496,0.626033,4.175687


In [21]:
len(predict({991},rules_fpgrowth))

0

In [22]:
for i in range(1811):
    if (len(predict({i},rules_fpgrowth))>0):
        print(i), print(predict({i},rules_fpgrowth))


122
    antecedents consequents   support  confidence       lift
101       (122)       (129)  0.004156    0.600000  18.219670
102       (122)       (406)  0.004849    0.700000  17.745515
103       (122)       (180)  0.004343    0.627027  17.480178
129
  antecedents consequents   support  confidence       lift
0       (129)       (406)  0.020594    0.625355  15.853217
133
    antecedents consequents   support  confidence       lift
368       (133)       (406)  0.017336    0.644398  16.335965
135
    antecedents consequents   support  confidence       lift
417       (135)       (406)  0.005729      0.6375  16.161094
136
   antecedents consequents   support  confidence       lift
81       (136)       (406)  0.007751    0.627273  15.901825
138
    antecedents consequents   support  confidence       lift
356       (138)       (129)  0.005635    0.603206  18.317037
357       (138)       (406)  0.006403    0.685371  17.374652
180
    antecedents consequents   support  confidence      lift
104

In [23]:
predict({122},rules_fpgrowth)

,antecedents,consequents,support,confidence,lift
101,(122),(129),0.004156,0.600000,18.219670
102,(122),(406),0.004849,0.700000,17.745515
103,(122),(180),0.004343,0.627027,17.480178


In [32]:
predict({1786},rules_fpgrowth)

,antecedents,consequents,support,confidence,lift
22661,(1786),(973),0.010840,0.716584,5.447712
22660,(1786),(1504),0.009398,0.621287,2.194949


In [25]:
preds = predict({979,857}, rules_fpgrowth)
preds

,antecedents,consequents,support,confidence,lift
21376,"(857, 979)","(1208, 873)",0.003857,0.611276,55.813153
21367,"(857, 979)",(1208),0.004662,0.738872,22.436686
21368,"(857, 979)",(873),0.004512,0.715134,10.456650
21369,"(857, 979)",(991),0.004381,0.694362,10.075700
21370,"(857, 979)",(866),0.004194,0.664688,7.926695
21371,"(857, 979)",(971),0.003969,0.629080,4.196015


In [26]:
rules_fpgrowth.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22694 entries, 21376 to 10804
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   antecedents  22694 non-null  object 
 1   consequents  22694 non-null  object 
 2   support      22694 non-null  float64
 3   confidence   22694 non-null  float64
 4   lift         22694 non-null  float64
dtypes: float64(3), object(2)
memory usage: 1.0+ MB


In [27]:
association_rules(freq_itemsets_fpgrowth, metric="confidence", min_threshold=0.7)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(184, 129)",(406),0.014884,0.039447,0.010765,0.723270,18.335438,0.010178,3.471091
1,"(184, 129, 180)",(406),0.009586,0.039447,0.007433,0.775391,19.656723,0.007054,4.276551
2,"(184, 180)",(406),0.015427,0.039447,0.011065,0.717233,18.182384,0.010456,3.396979
3,"(184, 562)",(406),0.008743,0.039447,0.006665,0.762313,19.325186,0.006320,4.041247
4,"(184, 562, 180)",(406),0.005617,0.039447,0.004549,0.810000,20.534096,0.004328,5.055544
...,...,...,...,...,...,...,...,...,...
11293,"(985, 1786, 1513)",(1504),0.004980,0.283053,0.004287,0.860902,3.041486,0.002878,5.154267
11294,"(1513, 1786, 973)",(985),0.005392,0.177276,0.003988,0.739583,4.171940,0.003032,3.159262
11295,"(985, 1786, 1513)",(973),0.004980,0.131539,0.003988,0.800752,6.087583,0.003333,4.358693
11296,"(1786, 972)",(1504),0.005092,0.283053,0.004100,0.805147,2.844509,0.002659,3.679425


In [37]:
rules_fpgrowth

,antecedents,consequents,support,confidence,lift
21376,"(857, 979)","(1208, 873)",0.003857,0.611276,55.813153
178,"(129, 186, 412)","(133, 406)",0.003819,0.641509,37.003871
157,"(412, 186, 180)","(133, 406)",0.004025,0.630499,36.368735
199,"(129, 186, 180)","(133, 406)",0.004643,0.600484,34.637437
200,"(186, 180, 133)","(129, 406)",0.004643,0.650919,31.607425
...,...,...,...,...,...
7835,"(977, 874, 1181)",(1504),0.006290,0.601073,2.123535
3374,"(1184, 977)",(1504),0.006534,0.600688,2.122176
9633,"(928, 977)",(1504),0.007433,0.600605,2.121881
4988,"(973, 1095)",(1504),0.005766,0.600390,2.121121
